In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'oxford_iiit_pet:4.*.*',
    split = ['train', 'test'],
    with_info = True,
    shuffle_files = True
)

In [3]:
num_classes = ds_info.features['label'].num_classes
print("Number of classes", num_classes)

Number of classes 37


In [4]:
IMAGE_SIZE = 224
def preprocess_and_augment(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)   # ← important change

    return image, label

def preprocess_test(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

In [5]:
def preprocess_test(image, label):
  image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  image = tf.cast(image, tf.float32)/255.0
  return image, label

In [6]:
ds_train = ds_train.map(lambda x: preprocess_and_augment(x['image'], x['label']), num_parallel_calls=tf.data.AUTOTUNE)
ds_test  = ds_test.map(lambda x: preprocess_test(x['image'], x['label']), num_parallel_calls=tf.data.AUTOTUNE)

In [7]:
BATCH_SIZE = 32
ds_train = ds_train.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test  = ds_test.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [8]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 37)             │        47,397 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,305,381 (8.79 MB)

 Trainable params: 47,397 (185.14 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [9]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    ds_train,
    epochs=30,
    validation_data=ds_test,
    callbacks=[early_stopping]
)

Epoch 1/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 57s 325ms/step - accuracy: 0.3432 - loss: 2.5709 - val_accuracy: 0.8201 - val_loss: 0.6561
Epoch 2/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 37s 67ms/step - accuracy: 0.8616 - loss: 0.4923 - val_accuracy: 0.8348 - val_loss: 0.5229
Epoch 3/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 69ms/step - accuracy: 0.8981 - loss: 0.3360 - val_accuracy: 0.8498 - val_loss: 0.4665
Epoch 4/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 68ms/step - accuracy: 0.9217 - loss: 0.2632 - val_accuracy: 0.8419 - val_loss: 0.4725
Epoch 5/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.9330 - loss: 0.2120 - val_accuracy: 0.8605 - val_loss: 0.4345
Epoch 6/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.9496 - loss: 0.1697 - val_accuracy: 0.8536 - val_loss: 0.4455
Epoch 7/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.9568 - loss: 0.1529 - val_accuracy: 0.8512 - val_loss: 0.4539
Epoch 8/30
115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 78ms/step - accuracy: 0.9647 - loss: 0.1270 - val

In [10]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low LR
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[early_stopping]
)

Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 79s 224ms/step - accuracy: 0.8224 - loss: 0.5603 - val_accuracy: 0.8613 - val_loss: 0.4206
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 18s 153ms/step - accuracy: 0.8863 - loss: 0.3938 - val_accuracy: 0.8654 - val_loss: 0.4210
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - accuracy: 0.8926 - loss: 0.3199 - val_accuracy: 0.8667 - val_loss: 0.4212
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 147ms/step - accuracy: 0.9126 - loss: 0.2894 - val_accuracy: 0.8651 - val_loss: 0.4276
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 146ms/step - accuracy: 0.9204 - loss: 0.2521 - val_accuracy: 0.8615 - val_loss: 0.4330
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 18s 161ms/step - accuracy: 0.9479 - loss: 0.1913 - val_accuracy: 0.8594 - val_loss: 0.4383
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - accuracy: 0.9502 - loss: 0.1767 - val_accuracy: 0.8602 - val_loss: 0.4441
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 148ms/step - accuracy: 0.9442 - loss: 0

Frontend

In [11]:
!pip install -q gradio

In [12]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

class_names = ds_info.features['label'].names

class_names = [name.replace('_', ' ').title() for name in class_names]

In [13]:
def predict_breed(image):

    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    image = tf.expand_dims(image, axis=0)

    predictions = model.predict(image)[0]
    top5_idx = np.argsort(predictions)[-5:][::-1]
    results = {class_names[i]: float(predictions[i]) for i in top5_idx}

    return results

In [14]:
# Create the interface
interface = gr.Interface(
    fn=predict_breed,
    inputs=gr.Image(type="pil", label="Upload a photo of a cat or dog"),
    outputs=gr.Label(num_top_classes=5, label="Predicted Breeds"),
    title="BreedNet",
    description="Upload a clear photo of a cat or dog to predict its breed. Powered by MobileNetV2 transfer learning on the Oxford IIIT Pet dataset.",
    examples=None,
    allow_flagging="never"


interface.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85d7ec2f97306b7902.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
